In [3]:
import argparse
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
import pandas as pd
from pandas import DataFrame
import json
from oauth2client import client
from oauth2client import file
from oauth2client import tools
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
#CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
CLIENT_SECRETS_PATH = 'client_secret_930859458171-ltc2m83ahbr8ovbl1h2q7spj8oadgi9p.apps.googleusercontent.com.json'
VIEW_ID = '77317038'
##VIEW_ID = '92320289'
##VIEW_ID= '168433959'

##****************************************************************************************##
def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])
  #flags = argparse.ArgumentParser(parents=[oauth2client.tools.argparser]).parse_args(["--noauth_local_webserver"])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))
  print(flow)

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  print(credentials)

  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics

##****************************************************************************************##

def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
		  'pageSize': 10000,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:pageViews'},{'expression': 'ga:users'},{'expression': 'ga:sessions'},{'expression': 'ga:pageviewsPerSession'},{'expression': 'ga:timeOnPage'},{'expression': 'ga:uniquePageviews'}],
		  'dimensions':[{'name':'ga:Date'},{'name':'ga:City'},{'name':'ga:channelGrouping'},{'name':'ga:landingPagePath'},{'name':'ga:pagePath'},{'name':'ga:exitPagePath'},{'name':'ga:secondPagePath'},{'name':'ga:medium'},{'name':'ga:browser'}]##,{'name':'ga:secondPagePath'},{'name':'ga:campaign'},{'name':'ga:source'},{'name':'ga:adGroup'}]
		}]
      }
  ).execute()

##****************************************************************************************##

def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response"""

  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])

    for row in rows:
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])

      for header, dimension in zip(dimensionHeaders, dimensions):
        print(header + ': ' + dimension)

      for i, values in enumerate(dateRangeValues):
        print('Date range (' + str(i) + ')')
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print(metricHeader.get('name') + ': ' + value)

##****************************************************************************************##

def get_first_profile_id(service):
  """Traverses Management API to return the first profile id.
  This first queries the Accounts collection to get the first account ID.
  This ID is used to query the Webproperties collection to retrieve the first
  webproperty ID. And both account and webproperty IDs are used to query the
  Profile collection to get the first profile id.
  Args:
    service: The service object built by the Google API Python client library.
  Returns:
    A string with the first profile ID. None if a user does not have any
    accounts, webproperties, or profiles.
  """

  accounts = service.management().accounts().list().execute()

  if accounts.get('items'):
    firstAccountId = accounts.get('items')[0].get('id')
    webproperties = service.management().webproperties().list(
        accountId=firstAccountId).execute()

    if webproperties.get('items'):
      firstWebpropertyId = webproperties.get('items')[0].get('id')
      profiles = service.management().profiles().list(
          accountId=firstAccountId,
          webPropertyId=firstWebpropertyId).execute()

      if profiles.get('items'):
        return profiles.get('items')[0].get('id')

  return None

##****************************************************************************************##

def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  #print(response)
  #print_response(response)
  return (response)
  #with open('\\usr\\hduser\\GA_Lastweek.json', 'w') as outfile:
    #json.dump(response,outfile)

##****************************************************************************************##

if __name__ == '__main__':
  dict_ga=main()
df_ga=pd.DataFrame(dict_ga)
df_ga['reports'] = df_ga['reports'].astype('str')
r = json.dumps(dict_ga)
loaded_r = json.loads(r)
lst = loaded_r['reports']

df_dimensions_metrics = DataFrame.from_records(lst[0]['data']['rows'])
df_dimensions = df_dimensions_metrics['dimensions']
df_metrics=df_dimensions_metrics['metrics']

##****************************************************************************************##

Date=[]
City=[]
channelGrouping=[]
landingPagePath=[]
pagePath=[]
exitPagePath=[]
secondPagePath=[]
medium=[]
browser=[]

pageViews=[]
users=[]
sessions=[]
pageviewsPerSession=[]
timeOnPage=[]
uniquePageviews=[]

for i in range(0, len(df_dimensions_metrics)):
    Date.append(df_dimensions[i][0])
    City.append(df_dimensions[i][1])
    channelGrouping.append(df_dimensions[i][2])
    landingPagePath.append(df_dimensions[i][3])
    pagePath.append(df_dimensions[i][4])
    exitPagePath.append(df_dimensions[i][5])
    secondPagePath.append(df_dimensions[i][6])
    medium.append(df_dimensions[i][7])
    browser.append(df_dimensions[i][8])
    
    pageViews.append(df_metrics[i][0]['values'][0])
    users.append(df_metrics[i][0]['values'][1])
    sessions.append(df_metrics[i][0]['values'][2])
    pageviewsPerSession.append(df_metrics[i][0]['values'][3])
    timeOnPage.append(df_metrics[i][0]['values'][4])
    uniquePageviews.append(df_metrics[i][0]['values'][5])

##****************************************************************************************##

df_all = pd.DataFrame(
    {
    'Date': Date,
    'City':City,
    'channelGrouping':channelGrouping,
    'landingPagePath':landingPagePath,
    'pagePath':pagePath,
    'exitPagePath':exitPagePath,
    'secondPagePath':secondPagePath,
    'medium':medium,
    'browser':browser,
    'users':users,
    'sessions':sessions,
    'pageviewsPerSession':pageviewsPerSession,
    'timeOnPage':timeOnPage,
    'uniquePageviews':uniquePageviews
    })

from pyspark.sql import SQLContext
from pyspark import SparkContext

try:
    sc.stop()
except:
    pass
sc = SparkContext("local", "GA_Lastweeks")

from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("GA_Lastweeks").getOrCreate()
df = sparkSession.createDataFrame(df_all)
#df.write.csv("hdfs://localhost:54310/user/hduser/test999.csv")
df.write.mode("overwrite").saveAsTable("default.GA_Lastweeks_test123")
